<a href="https://colab.research.google.com/github/Katsushi-Nakanishi/quant-macro/blob/main/GrowthAccounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np

pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')

In [31]:
#対象の国をリスト化
target_countries = ['Australia', 'Austria']

#リストに該当する国の、1960~2000期間のデータを抽出
data = pwt90[
    pwt90['country'].isin(target_countries) &
    pwt90['year'].between(1960, 2000)
]

#必要な変数だけをリスト化し、データを絞る
relevant_cols = ['countrycode', 'country', 'year', 'rgdpna', 'rkna', 'emp', 'avh', 'labsh',]
data = data[relevant_cols].dropna()

In [32]:
data['alpha'] = 1 - data['labsh']  # 資本所得分配率（α）= 1 - 労働所得分配率
data['L'] = data['emp'] * data['avh']  # L（総労働時間） = emp * avh
data['y'] = data['rgdpna'] / data['L']  # y = Y/L
data['k'] = data['rkna'] / data['L']  # k = K/L

In [35]:
data = data.sort_values('year')  # 年順にソート
data = data.groupby('countrycode').apply(lambda x:x.assign(
    y_shifted = 100 * x['y'] / x['y'].iloc[0],
    k_shifted = 100 * x['k'] / x['k'].iloc[0]
)).reset_index(drop=True).dropna()  # 国別に、最初の年で基準化したy,k列をデータに追加

<ipython-input-35-f0239f77adc4>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('countrycode').apply(lambda x:x.assign(


In [34]:
print(data)

   countrycode    country  year         rgdpna          rkna       emp  \
0          AUS  Australia  1960  174210.734375  5.488126e+05  4.014894   
1          AUS  Australia  1961  176586.343750  5.626911e+05  4.026486   
2          AUS  Australia  1962  187672.468750  5.850144e+05  4.123091   
3          AUS  Australia  1963  200342.343750  6.088022e+05  4.240949   
4          AUS  Australia  1964  213012.218750  6.400842e+05  4.393102   
..         ...        ...   ...            ...           ...       ...   
77         AUT    Austria  1996  258023.437500  1.126111e+06  3.575497   
78         AUT    Austria  1997  263712.906250  1.159388e+06  3.604259   
79         AUT    Austria  1998  273101.000000  1.193773e+06  3.640408   
80         AUT    Austria  1999  282905.281250  1.227244e+06  3.693053   
81         AUT    Austria  2000  292434.687500  1.263912e+06  3.727499   

            avh     labsh     alpha            L          y           k  \
0   2104.390218  0.687863  0.312137 

In [ ]:
import pandas as pd
import numpy as np

pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')

oecd_countries = [
    'United Kingdom', 'United States',
]

#リストに該当する国の、1970~2010の間のデータを抽出
data = pwt90[
    pwt90['country'].isin(oecd_countries) &
    pwt90['year'].between(1970, 2010)
]

relevant_cols = ['countrycode', 'country', 'year', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']
data = data[relevant_cols].dropna()

# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['y_n'] = data['rgdpna'] / data['emp']  # Y/N
data['hours'] = data['emp'] * data['avh']  # L
data['tfp_term'] = data['rtfpna'] ** (1 / (1 - data['alpha']))  # A^(1/(1-alpha))
data['cap_term'] = (data['rkna'] / data['rgdpna']) ** (data['alpha'] / (1 - data['alpha']))  # (K/Y)^(alpha/(1-alpha))
data['lab_term'] = data['hours'] / data['pop']  # L/N
data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
    alpha=1 - x['labsh'],
    y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],
    tfp_term_shifted=100 * x['tfp_term'] / x['tfp_term'].iloc[0],
    cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],
    lab_term_shifted=100 * x['lab_term'] / x['lab_term'].iloc[0]
)).reset_index(drop=True).dropna()


def calculate_growth_rates(country_data):

    start_year_actual = country_data['year'].min()
    end_year_actual = country_data['year'].max()

    start_data = country_data[country_data['year'] == start_year_actual].iloc[0]
    end_data = country_data[country_data['year'] == end_year_actual].iloc[0]

    years = end_data['year'] - start_data['year']

    g_y = ((end_data['y_n'] / start_data['y_n']) ** (1/years) - 1) * 100

    g_k = ((end_data['cap_term'] / start_data['cap_term']) ** (1/years) - 1) * 100

    g_a = ((end_data['tfp_term'] / start_data['tfp_term']) ** (1/years) - 1) * 100

    alpha_avg = (start_data['alpha'] + end_data['alpha']) / 2.0
    capital_deepening_contrib = alpha_avg * g_k
    tfp_growth_calculated = g_a

    tfp_share = (tfp_growth_calculated / g_y)
    cap_share = (capital_deepening_contrib / g_y)

    return {
        'Country': start_data['country'],
        'Growth Rate': round(g_y, 2),
        'TFP Growth': round(tfp_growth_calculated, 2),
        'Capital Deepening': round(capital_deepening_contrib, 2),
        'TFP Share': round(tfp_share, 2),
        'Capital Share': round(cap_share, 2)
    }


results_list = data.groupby('country').apply(calculate_growth_rates).dropna().tolist()
results_df = pd.DataFrame(results_list)

avg_row_data = {
    'Country': 'Average',
    'Growth Rate': round(results_df['Growth Rate'].mean(), 2),
    'TFP Growth': round(results_df['TFP Growth'].mean(), 2),
    'Capital Deepening': round(results_df['Capital Deepening'].mean(), 2),
    'TFP Share': round(results_df['TFP Share'].mean(), 2),
    'Capital Share': round(results_df['Capital Share'].mean(), 2)
}
results_df = pd.concat([results_df, pd.DataFrame([avg_row_data])], ignore_index=True)

print("\nGrowth Accounting in OECD Countries: 1970-2010 period")
print("="*85)
print(results_df.to_string(index=False))




Growth Accounting in OECD Countries: 1970-2010 period
       Country  Growth Rate  TFP Growth  Capital Deepening  TFP Share  Capital Share
United Kingdom         1.79        1.67              -0.11       0.93          -0.06
 United States         1.59        1.29               0.16       0.81           0.10
       Average         1.69        1.48               0.02       0.87           0.02


<ipython-input-1-974326e13e4f>:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
<ipython-input-1-974326e13e4f>:67: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_list = data.groupby('country').apply(calculate_growth_rates).dropna().tolist()
